# Experiment with finding circles in the Rett Girls videos

July 2018

This attempts using images over time.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
# %config InlineBackend.print_figure_kwargs={'bbox_inches':None}
import cv2
import numpy as np
import os.path as osp
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl

mpl.rcParams['figure.dpi']= 300

In [ ]:
vid = '/home/gb/Dropbox/Karen and Gary Shared Files/Videos & Transcripts/MSB/MSB_Video 1 (09-30-17).mp4'

osp.exists(vid)

In [ ]:
vc = cv2.VideoCapture(vid)
vc.get(cv2.CAP_PROP_FRAME_COUNT), vc.get(cv2.CAP_PROP_FPS), vc.get(cv2.CAP_PROP_FRAME_WIDTH), vc.get(cv2.CAP_PROP_FRAME_HEIGHT)

In [ ]:
def show(im, **kwargs):
    height, width = im.shape[:2]
    dpi = 100
    margin= 50
    figsize=((width+2*margin)/dpi, (height+2*margin)/dpi) # inches
    left = margin/dpi/figsize[0] #axes ratio
    bottom = margin/dpi/figsize[1]

    fig = plt.figure(figsize=figsize, dpi=dpi)
    fig.subplots_adjust(left=left, bottom=bottom, right=1.-left, top=1.-bottom)
    
    args = dict(kwargs)
    
    if len(im.shape) == 3:
        im = cv2.cvtColor(im.astype(np.uint8), cv2.COLOR_LAB2RGB)
    elif len(im.shape) == 2:
        print('here')
        args['cmap'] = 'gray'                  

    plt.imshow(im, **args)

In [ ]:
def grabFrame(fn):
    vc.set(cv2.CAP_PROP_POS_FRAMES, fn)
    rval, im = vc.read()
    im = cv2.cvtColor(im, cv2.COLOR_BGR2LAB)
    return im
show(grabFrame(785), interpolation=None)

In [ ]:
b1 = cv2.cvtColor(np.array([[[154,116,92]]], dtype=np.uint8), cv2.COLOR_LAB2RGB)
b2 = cv2.cvtColor(b1.astype(np.float32)/255, cv2.COLOR_RGB2LAB)
b2

In [ ]:
154*100/255, 116-128, 92-128, 10.4*100/255

In [ ]:
def isBlue(lim):
    mblue = np.array([154, 116, 92], dtype=np.float32)
    sblue = np.array([10.4, 3.2, 8.5], dtype=np.float32)
    blue = np.ones(lim.shape[:2], dtype=np.uint8)

    for i in range(3):
        blue &= np.abs(lim[:,:,i] - mblue[i]) < 2*sblue[i]

    return blue

show(isBlue(grabFrame(790)), cmap='gray')


Frames for page 4 begin at 683 and go to 844

Grab that stack.

In [ ]:
frames = np.array([grabFrame(f) for f in range(683,845)], dtype=np.float32)


In [ ]:
frames[0].shape

### Compute a standard deviation image


In [ ]:
s = np.std(frames[:,:,:,0], axis=0)
show(s)
plt.grid('on')

**Colors passing through a dot**

In [ ]:
plt.plot(frames[:,330,210,:])

In [ ]:
plt.imshow(frames[80,326:340,206:220,0], cmap='gray')
plt.figure()
plt.imshow(np.mean(frames[80:121,326:340,206:220,0], axis=0), cmap='gray')
plt.figure()
plt.imshow(np.std(frames[80:121,326:340,206:220,0], axis=0), cmap='gray')

In [ ]:
def MaxAbsChange(frames):
    return np.max(np.abs(np.diff(frames, axis=0)), axis=0)
show(MaxAbsChange(frames[:,:,:,0]))

In [ ]:
show(np.logical_or.reduce(np.abs(np.diff(frames[:,:,:,0], axis=0))>10, axis=0))

In [ ]:
def change(f1, f2):
    delta = np.abs(f1[:,:,0]-f2[:,:,0])
    return delta
f = 127
show(change(frames[f], frames[f+1]), cmap='gray')

In [ ]:
spots = (change(frames[77], frames[78]) > 20) & isBlue(frames[78])
spots = cv2.dilate(spots, np.ones((3,3), dtype=np.uint8), iterations=2)
spots = cv2.erode(spots, np.ones((3,3), dtype=np.uint8), iterations=2)
show(spots, cmap='gray')

In [ ]:
for f in range(len(frames)-1):
    spots = (change(frames[f], frames[f+1]) > 20) & isBlue(frames[f+1])
    spots = cv2.dilate(spots, np.ones((3,3), dtype=np.uint8), iterations=2)
    spots = cv2.erode(spots, np.ones((3,3), dtype=np.uint8), iterations=2)
    if np.sum(spots) > 10:
        print(f)
        show(spots, cmap='gray')

In [ ]:
show(frames[125])

In [ ]:
f = 125
changed = (change(frames[f], frames[f+1]) > 30)
show(changed, cmap='gray')
blue = isBlue(frames[f])
show(blue, cmap='gray')
spots = changed & blue
spots = cv2.dilate(spots, np.ones((3,3), dtype=np.uint8), iterations=2)
spots = cv2.erode(spots, np.ones((3,3), dtype=np.uint8), iterations=2)
show(spots, cmap='gray')

In [ ]:
show(frames[125])

In [ ]:
show(frames[126])

In [ ]:
im = frames[125]
def blueProb(im):
    mblue = np.array([154, 116, 92], dtype=np.float32)
    dblue = np.sqrt(np.sum((im - mblue)**2, axis=2))
    print(np.min(dblue), np.max(dblue))
    r = np.clip(1.0 - dblue / 40, 0.0, 1.0)
    print(np.min(r), np.max(r))
    return r
plt.imshow(blueProb(im), cmap='gray')

In [ ]:
def isBlue(lim):
    mblue = np.array([154, 116, 92], dtype=np.float32)
    sblue = np.array([10.4, 3.2, 8.5], dtype=np.float32)
    blue = np.ones(lim.shape[:2], dtype=np.uint8)

    for i in range(3):
        blue &= np.abs(lim[:,:,i] - mblue[i]) < 2*sblue[i]

    return blue


In [ ]:
lim = cv2.cvtColor(im, cv2.COLOR_RGB2LAB)
target = lim[429,70]
target

In [ ]:
error = np.sqrt(np.sum((lim.astype(np.float32) - target.astype(np.float32))**2, axis=2))
#error = 1.0 - error / np.max(error)
#error = np.clip((error - 0.8) / 0.1, 1, -1).astype(np.float32)
#error = error - np.mean(error)
#error = error / np.max(error)
error = np.where(error < 50, 1, -1)
error = error.astype(np.float32)

show(error[300:,:200], cmap='gray')
plt.grid('on')

## make a circular template with on center / off surround

In [ ]:
def makeTemplate(radius, delta, width=None):
    if width is None:
        width = np.ceil(radius * np.sqrt(2*np.pi))
    center = (width - 1) / 2
    d1 = (np.arange(width) - center) / radius
    x,y = np.meshgrid(d1, d1)
    dist = np.sqrt(x**2 + y**2)
    t = np.clip((dist - 1)/ delta, 1, -1).astype(np.float32)
    #t = t - np.mean(t)
    #t = t / np.sqrt(np.sum(t**2))
    #print(np.max(t))
    return t.astype(np.float32)
plt.figure()
plt.imshow(makeTemplate(5, 0.1), cmap='gray')

In [ ]:
template = makeTemplate(9, 0.1)

In [ ]:
np.max(template)

In [ ]:
res = cv2.matchTemplate(error, template, cv2.TM_CCORR)

In [ ]:
show(res)

In [ ]:
threshold = np.max(res) * 0.9
loc = np.where( res >= threshold)
for pt in zip(*loc[::-1]):
    print(pt)

In [ ]:
def overlap(p1, p2, r):
    return np.sum((p1 - p2)**2) < r**2

def getTheBest(res, threshold, radius, offset):
    yxs = np.argwhere(res > threshold)
    best = []
    for v, r1, c1 in sorted([(res[yx[0],yx[1]],yx[0],yx[1]) for yx in yxs], reverse=True):
        p1 = np.array([r1,c1]) + offset
        for p2 in best:
            if overlap(p1, p2, 2*radius):
                break
        else:
            best.append(p1)
    return np.array(best)

In [ ]:
radius = 8
template = makeTemplate(radius, 0.1)
ts2 = template.shape[0] // 2
#print(ts2)
res = cv2.matchTemplate(error, template, cv2.TM_CCORR)
threshold = np.max(res) * 0.5
plt.figure()
plt.grid('on')
plt.imshow(res[300:,:200], cmap='gray')
best = getTheBest(res, threshold, radius, template.shape[0]/2)
print(best)
orig = im.copy()
for pt in best:
    rc = tuple(pt[::-1].astype(int))
    cv2.circle(orig, rc, radius, (0,0,0), 1)
plt.figure()
plt.grid('on')
plt.imshow(orig[300:,:200])
plt.figure()
plt.grid('on')
plt.imshow(error[300:,:200])

In [ ]:


def findFixations(image, radius, tscale=0.5, slope=0.1):
    template = makeTemplate(radius, slope)
    target = np.array([154, 117,  87], dtype=np.float32) # special blue
    lim = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)

    error = np.sqrt(np.sum((lim.astype(np.float32) - target)**2, axis=2))
    error = np.where(error < 10, 1, -1)
    error = error.astype(np.float32)
    res = cv2.matchTemplate(error, template, cv2.TM_CCORR_NORMED)
    threshold = np.max(res) * tscale
    print(threshold)

    #show(res, cmap='gray')
    #plt.grid('on')
    best = getTheBest(res, threshold, radius, template.shape[0]/2)
    print(best)
    orig = image.copy()
    for pt in best:
        rc = tuple(pt[::-1].astype(int))
        cv2.circle(orig, rc, radius, (0,0,0), 1)

    show(orig)
    plt.grid('on')
    show(error)
    plt.grid('on')
    
    return best
    
vc.set(cv2.CAP_PROP_POS_FRAMES, 600)
rval, im = vc.read()
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
b = findFixations(im, 8)

In [ ]:
vc.set(cv2.CAP_PROP_POS_FRAMES, 600)
rval, im = vc.read()
im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
b = findFixations(im, 8)

image = im[180:225,280:325]
show(im)

In [ ]:
radius = 8
slope = 0.1

template = makeTemplate(radius, slope, width=24)
#template = template - np.mean(template)
plt.plot(template[radius,:])
np.sum(template - np.mean(template))

In [ ]:
tscale = 0.5 
target = np.array([154, 117,  87], dtype=np.float32) # special blue
lim = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)

error = np.sqrt(np.sum((lim.astype(np.float32) - target)**2, axis=2))
error = np.where(error < 10, 1, -1)
error = error.astype(np.float32)
res = cv2.matchTemplate(error, template, cv2.TM_CCORR_NORMED)
threshold = np.max(res) * tscale
print(threshold)

#show(res, cmap='gray')
#plt.grid('on')
best = getTheBest(res, threshold, radius, template.shape[0]/2)
print(best)
orig = image.copy()
for pt in best:
    rc = tuple(pt[::-1].astype(int))
    cv2.circle(orig, rc, radius, (0,0,0), 1)

show(orig)
plt.grid('on')
show(error)
plt.grid('on')
show(res, cmap='gray')
plt.grid('on')
 

In [ ]:
cv2.minMaxLoc(res)

In [ ]:
res[14,11]

In [ ]:
show(error[14:14+22,11:11+22])

In [ ]:
np.sum(error[14:14+22,11:11+22]*template)

In [ ]:
plt.imshow(error[14:14+22,11:11+22]*template)

In [ ]:
frames = []
for f in range(0, int(vc.get(cv2.CAP_PROP_FRAME_COUNT)), 1):
    vc.set(cv2.CAP_PROP_POS_FRAMES, f)
    rval, im = vc.read()
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    frames.append(im[40:60,-50:-20])
frames = np.array(frames)

In [ ]:
fig = plt.figure()
plt.imshow(frames.std(axis=0))

In [ ]:
plt.figure()
plt.figimage(im)

In [ ]:
# DPI, here, has _nothing_ to do with your screen's DPI.
dpi = 100.0
xpixels, ypixels = 854, 504

fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
ax = fig.add_axes([0, 0, 1, 1])
ax.axis('off')
ax.imshow(im)

In [ ]:
frames.shape